In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import torch
import torch_neuronx
from transformers import (
    AutoModel, AutoModelForSequenceClassification, AutoTokenizer  # Any other model class respective to the model we want to infer on
)

root: MASTER_ADDR environment variable is not set, defaulting to localhost
root: Found libneuronpjrt.so. Setting PJRT_DEVICE=NEURON.
/opt/conda/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [5]:
input_texts = ["""query: To run a model on Neuron with NxD Inference, you compile Python code into a NEFF file (Neuron Executable File Format), which you can load to Neuron devices using the Neuron Runtime.When you call compile(), NxD Inference does the following:

    Trace the Python code to produce an HLO file.

    Use the Neuron Compiler to compile the HLO file into a NEFF.

During the trace process, the model code is traced based on a given sample tensor for each input. As a result, model code should avoid dynamic logic that depends on the input values in a tensor, because NxD Inference compiles only the code path that is traced for the sample input tensor.

# Configure, initialize, and compile a model.
model = NeuronLlamaForCausalLM(model_path, config)
model.compile(compiled_model_path)"""]

tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-base')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-base')

batch_dict = tokenizer(input_texts, max_length=128, padding=True, truncation=True, return_tensors='pt')
batch_dict["input_ids"].shape, batch_dict["attention_mask"].shape

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


(torch.Size([1, 128]), torch.Size([1, 128]))

In [ ]:
traced = torch_neuronx.trace(model, (batch_dict['input_ids'], batch_dict['attention_mask']))
torch.jit.save(traced, "model.pt")

...
Compiler status PASS


In [6]:
neuron_model = torch.jit.load("model.pt")

In [9]:
neuron_model.eval()
neuron_model(batch_dict['input_ids'], batch_dict['attention_mask'])

{'last_hidden_state': tensor([[[ 0.5051,  0.9998,  0.2257,  ..., -0.6964, -0.7859,  0.1204],
          [ 0.3681,  1.2264,  0.1771,  ..., -0.8548, -0.7662,  0.4880],
          [ 0.3140,  1.2919,  0.1028,  ..., -0.9467, -0.7836,  0.3814],
          ...,
          [ 0.0610,  0.1019, -0.0209,  ..., -0.0102, -0.1298,  0.0549],
          [ 0.3243,  1.1426, -0.1265,  ..., -1.0515, -0.9133,  0.2963],
          [ 0.2262,  1.1364,  0.1295,  ..., -1.0918, -0.8646,  0.1324]]]),
 'pooler_output': tensor([[-5.0991e-01, -5.3726e-01, -2.5373e-01,  4.4081e-01, -4.8479e-01,
           3.8007e-01, -2.8186e-01, -4.7148e-01,  9.1544e-02,  1.8102e-01,
          -5.0594e-01, -1.6769e-02, -7.1494e-02, -2.2144e-01, -1.4783e-01,
           5.8110e-01,  3.8870e-01, -3.0914e-01, -2.4325e-01,  4.0434e-01,
           2.7061e-02,  3.1988e-01, -3.4744e-01, -3.5474e-01, -4.9449e-01,
           7.7209e-02,  1.3230e-01,  3.5306e-01,  1.2065e-01, -7.7349e-01,
          -3.0228e-02, -6.9486e-01,  1.5407e-01, -2.3809e-01, 

In [10]:
%%time
for i in range(1000):
    batch_dict = tokenizer(input_texts, max_length=128, padding=True, truncation=True, return_tensors='pt')
    neuron_model(batch_dict['input_ids'], batch_dict['attention_mask'])

CPU times: user 600 ms, sys: 122 ms, total: 721 ms
Wall time: 1.47 s
